In [1]:
import numpy as np 
from scipy.stats import multivariate_normal
import Estimateurs

On commence par tirer le $\theta$* ($\in \mathbb{R}$) qui sera le paramètre que l'on cherchera à estimer par la suite. 

A la manière de la génération des données dans l'article Rainforth et al. (2018) : $\theta^* \sim \mathcal{N}(0, 1)$

In [2]:
#theta_true = np.random.multivariate_normal(np.zeros(20), np.identity(20))
theta_true = np.random.normal(0, 1)
print(f"La valeur de theta à estimer est {int(theta_true*100)/100}")

La valeur de theta à estimer est 1.01


Désormais, on va tirer un échantillon $((Z_i, X_i))_i \stackrel{iid}{\sim} \mathcal{N}(\boldsymbol{z}|\theta, \boldsymbol{I}) * \mathcal{N}(\boldsymbol{x}|\boldsymbol{z}, \boldsymbol{I})$. 


In [3]:
## On se donne notre échantillon x

x, _ = Estimateurs.joint_probability(theta_true)

print(f"L'échantillon x observé est :\n \n x = {x} \n \n et sa taille est {x.shape}")

L'échantillon x observé est :
 
 x = [ 0.62708049 -1.88402291  0.27970445  2.91995055  3.35245272  3.03822922
 -1.32046595  0.10037036 -1.40654483  2.88937144 -1.9155619   2.87865627
  2.37488217  0.66437847  1.50190432  2.29082669  2.17680317  1.97622803
 -0.01341003  0.94014867] 
 
 et sa taille est (20,)


Ceci nous suffit pour tester notre classe Estimateurs dont le code se trouve dans le fichier Estimateurs.py

In [4]:
theta_hat = np.mean(x)
A, b = Estimateurs.noised_params((1/2)*np.eye(20), (np.zeros(20) + theta_true)/2)
A_hat, b_hat = Estimateurs.noised_params((1/2)*np.eye(20), (np.zeros(20) + theta_hat)/2)

In [5]:
z_sample = Estimateurs.generate_encoder(x, 4, A, b)[0]

def compute_ratio(x, z, theta, A, b):
    # Parameters
    I = np.eye(20)

    # Probability densities
    p_theta_xz = np.exp(-0.5 * np.dot(np.dot((z - theta).T, np.linalg.inv(2*I)), (z - theta)) - 0.5 * np.dot(np.dot((x - z).T, np.linalg.inv(I)), (x - z)))
    q_phi_z_given_x_density = np.exp(-0.5 * np.dot(np.dot((z - (np.dot(A, x) + b)).T, np.linalg.inv((2/3)*I)), (z - (np.dot(A, x) + b.flatten()))))

    # Compute the ratio
    ratio = p_theta_xz / q_phi_z_given_x_density

    return ratio

def weight(x, theta_hat, theta, A_hat, b_hat, A, b, z):

    
    ## calcul des poids pour theta_hat (associé à l'observation x)
    weight_theta_hat = compute_ratio(x, z, theta_hat, A_hat, b_hat)

    ## cacul des poids pour theta (associé à un theta)                                                                                                    
    weight_theta = compute_ratio(x, z, theta, A, b)
    
    return (weight_theta_hat, weight_theta) ## attention, renvoie un tuple
              
def l_hat(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample): 


    ## calcul de l_theta_hat (pour theta_hat)
    l_theta_hat = np.log((1/(len(z_sample)) * sum(weight(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample[i])[0] for i in range(len(z_sample)))))

    ## calcul de l_theta (pour theta)
    l_theta = np.log((1/(len(z_sample)) * sum(weight(x, theta_hat, theta, A_hat, b_hat, A, b, z_sample[i])[1] for i in range(len(z_sample)))))

    return (l_theta_hat, l_theta)

In [6]:
l_hat(z_sample[0], theta_hat, theta_true, A_hat, b_hat, A, b, z_sample)

(-2.8827480076217213, -2.91285760298495)

In [7]:
estimateur = Estimateurs.Estimateurs(x, theta_true, 0.6)

In [8]:
x = estimateur.grad_ML_RR(theta_star=theta_true, n_simulations=1)

In [10]:
x

array([-85.33360645, -20.90572445,  28.96190213,   4.58250994,
        -6.06496283,   6.06055485,  -5.04536984, -40.52988114,
       -31.48126449,   6.12631459,  13.39693739,  25.40134969,
        20.86514062, -20.95379378,   3.98967579,  33.86274255,
         8.76821409,  37.96856848,  44.81163528,   7.84492834])